In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
import os
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
#tokenization for wordcloud, ect..
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tylergehbauer/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
sony_df = pd.read_csv("../data/sony.csv")

In [3]:
sentiments_sony = []#creates empty list

#Going to use VADER’s SentimentIntensityAnalyzer() to see how many 'Neutral (neu)' , 'Positive (pos)', and 'Negatvie(neg)'
# words there are for each article. 
for articles in sony_df['critics_consensus']: #goes through each artilce in df and applies polarity
    try:
        text = articles #accessing each articles 'content' which contains the article itself in 'bitcoin_news'
        results = analyzer.polarity_scores(text) #this is how get compound, pos, neg, and nue polarity scores
        #during imports we set analyzer equal to SentimentIntensityAnalyzer() 
        #'results' uses this function to find polarity score for each article in bitcoin_news
        compound = results["compound"] #gets compound polarity score
        pos = results["pos"] #gets postive polarity score
        neu = results["neu"] #gets neutral polarity score
        neg = results["neg"] #gets negative polarity score

        sentiments_sony.append({
            "Consensus": text,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
        }) #appends a dictionary inside the list-(sentiments) so a dataframe can be created
        #this appends all polarity score we got above
    except AttributeError:
        pass
#rinse and repeat for each article    
sony = pd.DataFrame(sentiments_sony)#creates a dataframe using the list of dicts
sony = sony.join(sony_df['movie_title'])
sony = sony.join(sony_df['original_release_date'])
sony = sony.join(sony_df['streaming_release_date'])
sony = sony.drop(columns = ['Positive', 'Negative', 'Neutral'])

In [4]:
sony[["original_release_date", "streaming_release_date"]] = sony[["original_release_date", "streaming_release_date"]].apply(pd.to_datetime)


In [5]:
sony.head()

,text,Compound,Positive,Negative,Neutral,movie_title,original_release_date,streaming_release_date
0,Nicole Holofcener's newest might seem slight i...,0.5023,0.115,0.000,0.885,Please Give,2010-04-30,2012-09-04
1,An incomprehensible and forgettable sci-fi thr...,0.1027,0.113,0.000,0.887,Ultraviolet,2006-03-03,2012-04-16
2,Polanski's version of Dickens' classic won't h...,-0.3804,0.000,0.105,0.895,Oliver Twist,2005-09-30,2012-04-16
3,Broderick Crawford is spellbinding as politici...,0.3182,0.076,0.000,0.924,The Second Chance,2006-02-17,2006-07-18
4,Captivating performances from Steve Buscemi an...,0.4404,0.172,0.000,0.828,All the King's Men,1949-11-08,2013-05-01
